In [3]:
%load_ext autoreload
%autoreload 2

import sys
import os

try: # When on google Colab, let's clone the notebook so we download the cache.
    import google.colab
    repo_path = 'dspy'
    !git -C $repo_path pull origin || git clone https://github.com/stanfordnlp/dspy $repo_path
except:
    repo_path = '.'

if repo_path not in sys.path:
    sys.path.append(repo_path)


import pkg_resources # Install the package if it's not installed
if not "dspy-ai" in {pkg.key for pkg in pkg_resources.working_set}:
    !pip install -U pip
    !pip install dspy-ai
    # !pip install -e $repo_path

import dspy

fatal: cannot change to 'dspy': No such file or directory
Cloning into 'dspy'...
remote: Enumerating objects: 20976, done.
remote: Counting objects: 100% (1499/1499), done.
remote: Compressing objects: 100% (415/415), done.
remote: Total 20976 (delta 1281), reused 1091 (delta 1084), pack-reused 19477
Receiving objects: 100% (20976/20976), 21.18 MiB | 18.96 MiB/s, done.
Resolving deltas: 100% (7714/7714), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [4]:
turbo = dspy.OpenAI(model='gpt-3.5-turbo', api_key='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')

colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
dspy.settings.configure(lm=turbo, rm=colbertv2_wiki17_abstracts)

In [5]:
from dspy.datasets import HotPotQA
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7405 [00:00<?, ? examples/s]

In [6]:
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

In [7]:
class BasicQA(dspy.Signature):
  question = dspy.InputField()
  answer = dspy.OutputField(desc='min 10 words')

In [8]:
generate_answer = dspy.Predict(BasicQA)
p = generate_answer()

print(devset[10])
print(p.answer)

AuthenticationError: ignored

In [10]:
turbo.inspect_history(n=1)





Follow the following format.

Question: ${question}
Answer: min 10 words Question: What is the capital of France?
Answer: The capital of France is Paris.





In [46]:
generate_answers_with_chain_of_thought = dspy.ChainOfThought(BasicQA)
pred = generate_answers_with_chain_of_thought(question=devset[0].question)
print(pred)


Prediction(
    rationale='determine if both Cangzhou and Qionghai are in the Hebei province of China.',
    answer='No, Cangzhou is in Hebei province while Qionghai is in Hainan province.'
)


In [47]:
turbo.inspect_history(n=1)





Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: min 10 words

---

Question: Are both Cangzhou and Qionghai in the Hebei province of China?
Reasoning: Let's think step by step in order to determine if both Cangzhou and Qionghai are in the Hebei province of China.
Answer: No, Cangzhou is in Hebei province while Qionghai is in Hainan province.





In [18]:
retrieve = dspy.Retrieve(k=3)
topK_passages = retrieve(devset[0].question).passages

print(f"Top {retrieve.k} passages for question: {devset[0].question} \n", '-' * 30, '\n')

for idx, passage in enumerate(topK_passages):
    print(f'{idx+1}]', passage, '\n')

Top 3 passages for question: Are both Cangzhou and Qionghai in the Hebei province of China? 
 ------------------------------ 

1] Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People's Republic of China. At the 2010 census, Cangzhou's built-up ("or metro") area made of Yunhe, Xinhua districts and Cang County largely being conurbated had a population of 1,205,814 inhabitants, while the prefecture-level administrative unit in total has a population of 7,134,062. It lies approximately 90 km from the major port city of Tianjin, and 180 km from Beijing. 

2] Haixing County | Haixing County () is a county of southeastern Hebei province, China, bordering Shandong to the southeast. It is administered by Cangzhou City, and, , had a population of 220,000 residing in an area of 836 km2 . Both G18 Rongcheng–Wuhai Expressway and G25 Changchun–Shenzhen Expressway pass through the county. 

3] Dongguang County | Dongguang County () is a county under the jurisdiction of 

In [19]:
retrieve("When was the first FIFA World Cup held?")

Prediction(
    passages=['History of the FIFA World Cup | The FIFA World Cup was first held in 1930, when FIFA president Jules Rimet decided to stage an international football tournament. The inaugural edition, held in 1930, was contested as a final tournament of only thirteen teams invited by the organization. Since then, the World Cup has experienced successive expansions and format remodeling to its current 32-team final tournament preceded by a two-year qualifying process, involving over 200 teams from around the world.', "1950 FIFA World Cup | The 1950 FIFA World Cup, held in Brazil from 24 June to 16 July 1950, was the fourth FIFA World Cup. It was the first World Cup since 1938, the planned 1942 and 1946 competitions having been cancelled owing to World War II. It was won by Uruguay, who had won the inaugural competition in 1930, clinching the cup by beating the hosts Brazil 2–1 in the deciding match of the four-team final group (this was the only tournament not decided by a on

In [22]:
class GenerateAnswer(dspy.Signature):
  context = dspy.InputField(desc='may contain relevant facts')
  question = dspy.InputField()
  answer = dspy.OutputField(desc='b/w 1 and 5 words')

In [23]:
class RAG(dspy.Module):

  def __init__(self, num_passages=3):
    super().__init__()
    self.retrieve = dspy.Retrieve(k=num_passages)
    self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

  def forward(self, question):
    context = self.retrieve(question).passages
    predictions = self.generate_answer(context=context, question=question)
    return dspy.Prediction(context=context, answer=predictions.answer)


In [24]:
from dspy.teleprompt import BootstrapFewShot

def validate_context_and_answer(example, pred, trace=None):
  answer_EM = dspy.evaluate.answer_exact_match(example, pred)
  answer_PM = dspy.evaluate.answer_passage_match(example, pred)
  return answer_EM and answer_PM

teleprompter = BootstrapFewShot(metric=validate_context_and_answer)

compiled_rag = teleprompter.compile(RAG(), trainset=trainset)

 15%|█▌        | 3/20 [00:10<00:55,  3.29s/it]INFO:backoff:Backing off request(...) for 0.8s (openai.error.RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-Fli4NWOUzLm0bJ4z2w0EwOSB on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.)


Backing off 0.8 seconds after 1 tries calling function <function GPT3.request at 0x78acc6546b90> with kwargs {}


INFO:backoff:Backing off request(...) for 2.0s (openai.error.RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-Fli4NWOUzLm0bJ4z2w0EwOSB on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.)


Backing off 2.0 seconds after 2 tries calling function <function GPT3.request at 0x78acc6546b90> with kwargs {}


INFO:backoff:Backing off request(...) for 1.8s (openai.error.RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-Fli4NWOUzLm0bJ4z2w0EwOSB on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.)


...

Backing off 9.2 seconds after 5 tries calling function <function GPT3.request at 0x78acc6546b90> with kwargs {}


INFO:backoff:Backing off request(...) for 18.8s (openai.error.RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-Fli4NWOUzLm0bJ4z2w0EwOSB on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.)


Backing off 18.8 seconds after 6 tries calling function <function GPT3.request at 0x78acc6546b90> with kwargs {}


 55%|█████▌    | 11/20 [02:43<02:13, 14.84s/it]

Bootstrapped 4 full traces after 12 examples in round 0.


In [48]:
my_question = "What castle did David Gregory inherit?"
pred = compiled_rag(my_question)
print(pred)

Prediction(
    context=['David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinnairdy Castle in 1664. Three of his twenty-nine children became mathematics professors. He is credited with inventing a military cannon that Isaac Newton described as "being destructive to the human species". Copies and details of the model no longer exist. Gregory\'s use of a barometer to predict farming-related weather conditions led him to be accused of witchcraft by Presbyterian ministers from Aberdeen, although he was never convicted.', 'Gregory Tarchaneiotes | Gregory Tarchaneiotes (Greek: Γρηγόριος Ταρχανειώτης , Italian: "Gregorio Tracanioto" or "Tracamoto" ) was a "protospatharius" and the long-reigning catepan of Italy from 998 to 1006. In December 999, and again on February 2, 1002, he reinstituted and confirmed the possessions of the abbey and monks of 

In [49]:
turbo.inspect_history(n=1)





Question: At My Window was released by which American singer-songwriter?
Answer: John Townes Van Zandt

Question: "Everything Has Changed" is a song from an album released under which record label ?
Answer: Big Machine Records

Question: The Victorians - Their Story In Pictures is a documentary series written by an author born in what year?
Answer: 1950

Question: Which Pakistani cricket umpire who won 3 consecutive ICC umpire of the year awards in 2009, 2010, and 2011 will be in the ICC World Twenty20?
Answer: Aleem Sarwar Dar

Question: Having the combination of excellent foot speed and bat speed helped Eric Davis, create what kind of outfield for the Los Angeles Dodgers?
Answer: "Outfield of Dreams"

Question: Who is older, Aleksandr Danilovich Aleksandrov or Anatoly Fomenko?
Answer: Aleksandr Danilovich Aleksandrov

Question: The Organisation that allows a community to influence their operation or use and to enjoy the benefits arisingwas founded in what year?
Answer: 2010

Ques

In [51]:
my_question = "What was Rosario Dawson's first film?"
pred = compiled_rag(my_question)
print(pred)

Prediction(
    context=['Rosario Dawson | Rosario Isabel Dawson (born May 9, 1979) is an American actress, producer, singer, comic book writer, and political activist. She made her film debut in the 1995 teen drama "Kids". Her subsequent film roles include "He Got Game", "Men in Black II", "25th Hour", "Rent", "Sin City", "Death Proof", "Seven Pounds", "", and "Top Five". Dawson has also provided voice-over work for Disney and DC.', 'Light It Up (film) | Light It Up is a 1999 American hostage crime teen drama film starring an ensemble cast that consists of R&B singer/actor Usher Raymond (in his first leading role), Rosario Dawson, Forest Whitaker, and Vanessa L. Williams. The film was written and directed by Craig Bolotin, and produced by Kenneth "Babyface" Edmonds and his wife Tracy Edmonds.', 'Zookeeper (film) | Zookeeper is a 2011 American comedy film directed by Frank Coraci, starring Kevin James, Rosario Dawson and Leslie Bibb, and featuring the voices of Nick Nolte, Sylvester St

In [52]:
my_question = "What is Judi Dench's middle name and profession?"
pred = compiled_rag(my_question)
print(pred)

Prediction(
    context=['Judi Dench | Dame Judith Olivia "Judi" Dench, {\'1\': ", \'2\': ", \'3\': ", \'4\': "} (born 9 December 1934) is an English actress and author. Dench made her professional debut in 1957 with the Old Vic Company. Over the following few years, she performed in several of Shakespeare\'s plays in such roles as Ophelia in "Hamlet", Juliet in "Romeo and Juliet", and Lady Macbeth in "Macbeth". Although most of her work during this period was in theatre, she also branched into film work and won a BAFTA Award as Most Promising Newcomer. She drew strong reviews for her leading role in the musical "Cabaret" in 1968.', 'Judi Dench filmography | Dame Judi Dench is an English actress who has worked in theater, television, and film. Dench made her professional debut in 1957 with the Old Vic Company. Over the following few years she played in several of Shakespeare\'s plays in such roles as Ophelia in "Hamlet", Juliet in "Romeo and Juliet" and Lady Macbeth in "Macbeth". She b

In [29]:
for name, parameter in compiled_rag.named_predictors():
  print(f"{name} \n {parameter.demos}")

generate_answer 
 [Example({'augmented': True, 'context': ['Tae Kwon Do Times | Tae Kwon Do Times is a magazine devoted to the martial art of taekwondo, and is published in the United States of America. While the title suggests that it focuses on taekwondo exclusively, the magazine also covers other Korean martial arts. "Tae Kwon Do Times" has published articles by a wide range of authors, including He-Young Kimm, Thomas Kurz, Scott Shaw, and Mark Van Schuyver.', "Kwon Tae-man | Kwon Tae-man (born 1941) was an early Korean hapkido practitioner and a pioneer of the art, first in Korea and then in the United States. He formed one of the earliest dojang's for hapkido in the United States in Torrance, California, and has been featured in many magazine articles promoting the art.", 'Hee Il Cho | Cho Hee Il (born October 13, 1940) is a prominent Korean-American master of taekwondo, holding the rank of 9th "dan" in the martial art. He has written 11 martial art books, produced 70 martial art 

In [34]:
compiled_rag("name a battle fought by germans")

Prediction(
    context=['Defence of the Reich | The Defence of the Reich (German: "Reichsverteidigung" ) is the name given to the strategic defensive aerial campaign fought by the "Luftwaffe" over German-occupied Europe and Nazi Germany during World War II. Its aim was to prevent the destruction of German civilians, military and civil industries by the Western Allies. The day and night air battles over Germany during the war involved thousands of aircraft, units and aerial engagements to counter the Allied strategic bombing campaign. The campaign was one of the longest in the history of aerial warfare and with the Battle of the Atlantic and the Allied Blockade of Germany was the longest of the war. The "Luftwaffe" fighter force defended the airspace of German-occupied territory against attack, first by RAF Bomber Command and then against the United States Army Air Forces (USAAF).', 'Battle of Greece | The Battle of Greece (also known as Operation "Marita", German: "Unternehmen Marita"

In [35]:
from dspy.evaluate.evaluate import Evaluate

evaluate_on_hotpotqa = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)
metric = dspy.evaluate.answer_exact_match
evaluate_on_hotpotqa(compiled_rag, metric=metric)

Average Metric: 1 / 3  (33.3):   6%|▌         | 3/50 [00:07<01:56,  2.48s/it]INFO:backoff:Backing off request(...) for 0.3s (openai.error.RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-Fli4NWOUzLm0bJ4z2w0EwOSB on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.)


Backing off 0.3 seconds after 1 tries calling function <function GPT3.request at 0x78acc6546b90> with kwargs {}


INFO:backoff:Backing off request(...) for 1.9s (openai.error.RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-Fli4NWOUzLm0bJ4z2w0EwOSB on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.)


Backing off 1.9 seconds after 2 tries calling function <function GPT3.request at 0x78acc6546b90> with kwargs {}


INFO:backoff:Backing off request(...) for 3.4s (openai.error.RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-Fli4NWOUzLm0bJ4z2w0EwOSB on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.)


...

INFO:backoff:Backing off request(...) for 97.8s (openai.error.RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-Fli4NWOUzLm0bJ4z2w0EwOSB on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 7m12s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.)


Backing off 97.8 seconds after 10 tries calling function <function GPT3.request at 0x78acc6546b90> with kwargs {}


INFO:backoff:Backing off request(...) for 723.6s (openai.error.RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-Fli4NWOUzLm0bJ4z2w0EwOSB on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 7m12s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.)


Backing off 723.6 seconds after 11 tries calling function <function GPT3.request at 0x78acc6546b90> with kwargs {}


KeyboardInterrupt: ignored

In [38]:
turbo.inspect_history(n=1)





Question: At My Window was released by which American singer-songwriter?
Answer: John Townes Van Zandt

Question: "Everything Has Changed" is a song from an album released under which record label ?
Answer: Big Machine Records

Question: The Victorians - Their Story In Pictures is a documentary series written by an author born in what year?
Answer: 1950

Question: Which Pakistani cricket umpire who won 3 consecutive ICC umpire of the year awards in 2009, 2010, and 2011 will be in the ICC World Twenty20?
Answer: Aleem Sarwar Dar

Question: Having the combination of excellent foot speed and bat speed helped Eric Davis, create what kind of outfield for the Los Angeles Dodgers?
Answer: "Outfield of Dreams"

Question: Who is older, Aleksandr Danilovich Aleksandrov or Anatoly Fomenko?
Answer: Aleksandr Danilovich Aleksandrov

Question: The Organisation that allows a community to influence their operation or use and to enjoy the benefits arisingwas founded in what year?
Answer: 2010

Ques

In [36]:
class GenerateSearchQuery(dspy.Signature):
  context = dspy.InputField(desc='may contain relevant facts')
  question = dspy.InputField(desc='question to be answered as accurately as possible')
  query = dspy.OutputField(desc='Give detailed answers with proper explanation')

In [37]:
from dsp.utils import deduplicate

class SimplifiedBaleen(dspy.Module):
    def __init__(self, passages_per_hop=3, max_hops=2):
        super().__init__()

        self.generate_query = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.max_hops = max_hops

    def forward(self, question):
        context = []

        for hop in range(self.max_hops):
            query = self.generate_query[hop](context=context, question=question).query
            passages = self.retrieve(query).passages
            context = deduplicate(context + passages)

        pred = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=pred.answer)

In [39]:
my_question = "How many storeys are in the castle that David Gregory inherited?"
uncompiled_baleen = SimplifiedBaleen()
pred = uncompiled_baleen(my_question)
print(pred)

INFO:backoff:Backing off request(...) for 1.0s (openai.error.RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-Fli4NWOUzLm0bJ4z2w0EwOSB on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 7m12s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.)


Backing off 1.0 seconds after 1 tries calling function <function GPT3.request at 0x78acc6546b90> with kwargs {}


INFO:backoff:Backing off request(...) for 0.3s (openai.error.RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-Fli4NWOUzLm0bJ4z2w0EwOSB on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 7m12s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.)


Backing off 0.3 seconds after 2 tries calling function <function GPT3.request at 0x78acc6546b90> with kwargs {}


INFO:backoff:Backing off request(...) for 1.7s (openai.error.RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-Fli4NWOUzLm0bJ4z2w0EwOSB on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.)


...

INFO:backoff:Backing off request(...) for 2.1s (openai.error.RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-Fli4NWOUzLm0bJ4z2w0EwOSB on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.)


Backing off 2.1 seconds after 4 tries calling function <function GPT3.request at 0x78acc6546b90> with kwargs {}


INFO:backoff:Backing off request(...) for 13.3s (openai.error.RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-Fli4NWOUzLm0bJ4z2w0EwOSB on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.)


Backing off 13.3 seconds after 5 tries calling function <function GPT3.request at 0x78acc6546b90> with kwargs {}


KeyboardInterrupt: ignored